In [8]:
# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np


# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Realizar los distintos test
# ------------------------------------------------------------------------------

from scipy.stats import ttest_ind, norm, chi2_contingency, f_oneway
from sklearn.linear_model import LinearRegression


# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

In [9]:
df_ab_testing = pd.read_csv('df_transformado_limpio.csv')
df_ab_testing.sample(5)

,employeenumber,age,attrition,businesstravel,dailyrate,distancefromhome,education,educationfield,environmentsatisfaction,gender,hourlyrate,jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyincome,monthlyrate,numcompaniesworked,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearssincelastpromotion,yearswithcurrmanager,datebirth,remotework
978,8970,53,No,travel rarely,868.0,8,3,marketing,1,Male,73,3,4,sales executive,4,married,48620.0,22789,5,No,14,3.0,3,1,28.0,3,3.0,2,2,2,1970,Yes
218,3810,35,No,travel rarely,1315.0,22,3,life sciences,2,Female,71,4,3,manager,2,divorced,48620.0,19100,7,No,18,3.2,2,1,10.0,6,2.0,7,6,2,1988,No
52,19370,38,No,travel rarely,1394.0,8,3,life sciences,4,Female,58,2,2,research scientist,2,divorced,48620.0,18115,1,No,16,3.0,3,1,20.0,3,3.0,20,0,7,1985,No
58,3740,55,No,travel rarely,452.0,1,3,medical,4,Male,81,3,5,manager,1,single,190450.0,18938,0,No,14,3.0,3,0,37.0,2,3.0,36,4,13,1968,Yes
883,10610,50,No,travel rarely,1464.0,12,4,medical,12,Male,62,3,5,research director,3,married,192370.0,12853,2,No,11,3.0,4,1,29.0,2,2.0,8,7,7,1973,No


`Fase 4: Problema de A/B Testing.`

El objetivo de esta fase es determinar si existe una relación entre el nivel de satisfacción en el trabajo y la rotación de empleados, y si es así, cuál es la magnitud de esa relación. Partiremos de la siguiente hipótesis, "Existe una relación entre el nivel de satisfacción en el trabajo y la rotación de empleados en la empresa. Se sospecha que los empleados con niveles de satisfacción más bajos tienen una mayor probabilidad de dejar la empresa". Para ello deberéis crear una columna nueva en base al nivel de satisfacción en el trabajo para crear dos grupos, las condiciones para crear los grupos son:

- `Grupo A (Control)`: Empleados con un nivel de satisfacción en el trabajo igual o superior a 3 en una escala de 1 a 5.

- `Grupo B (Variante)`: Empleados con un nivel de satisfacción en el trabajo inferior a 3 en la misma escala.

La `métrica principal` que debes usar es la tasa de rotación de empleados (Attrition) en cada grupo. Recordemos que, en esta columna, "No" indica que el empleado no ha dejado la empresa (sin rotación), mientras que "Yes" indica que el empleado ha dejado la empresa (con rotación).

Por lo tanto, los pasos que deberás seguir en esta fase son:

1. Divide a los empleados en los grupos A y B según los criterios establecidos.

2. Calcula la tasa de rotación (porcentaje de empleados que dejaron la empresa) en cada grupo.

3. Realiza un análisis estadístico para determinar si hay una diferencia significativa en la tasa de rotación entre los grupos A y B.

4. Analiza los resultados.

5. Calcular la magnitud de esta relación utilizando estadísticas como la diferencia de medias por ejemplo.

In [22]:
# Crear la columna 'group' usando where 

df_ab_testing['group_AB_Testing'] = df_ab_testing['jobsatisfaction'].where(df_ab_testing['jobsatisfaction'] >= 3, 'A').where(df_ab_testing['jobsatisfaction'] < 3, 'B')

In [23]:
df_ab_testing.head(1)

,employeenumber,age,attrition,businesstravel,dailyrate,distancefromhome,education,educationfield,environmentsatisfaction,gender,hourlyrate,jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyincome,monthlyrate,numcompaniesworked,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearssincelastpromotion,yearswithcurrmanager,datebirth,remotework,group_AB_Testing
0,1620,51,No,travel rarely,684.0,6,3,life sciences,1,Male,51,3,5,research director,3,married,195370.0,6462,7,No,13,3.0,3,0,10.0,5,3.0,20,15,15,1972,Yes,B


In [36]:
# Filtrar el DataFrame para obtener solo los datos del grupo 'A'
grupo_a = df_ab_testing[df_ab_testing['group_AB_Testing'] == 'A']
grupo_a

,employeenumber,age,attrition,businesstravel,dailyrate,distancefromhome,education,educationfield,environmentsatisfaction,gender,hourlyrate,jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyincome,monthlyrate,numcompaniesworked,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearssincelastpromotion,yearswithcurrmanager,datebirth,remotework,group_AB_Testing
5,810,59,No,travel rarely,1435.0,25,3,life sciences,1,Female,99,3,3,sales executive,1,married,48620.0,2354,7,No,11,3.0,4,0,28.0,3,2.0,21,7,9,1964,Yes,A
11,8250,58,Yes,travel rarely,286.0,2,4,life sciences,4,Male,31,3,5,research director,2,single,192460.0,25761,7,Yes,12,3.0,4,0,40.0,2,3.0,31,13,8,1965,No,A
15,12940,36,No,travel rarely,429.0,2,4,life sciences,3,Female,53,3,2,manufacturing director,2,single,48620.0,2323,9,Yes,11,3.0,4,0,18.0,2,3.0,16,5,12,1987,No,A
20,1000,35,No,travel rarely,776.0,1,4,marketing,3,Male,32,2,2,sales executive,1,single,48620.0,23016,0,No,14,3.2,2,0,16.0,2,3.0,15,2,8,1988,No,A
21,2440,46,No,travel rarely,526.0,1,2,life sciences,2,Female,92,3,3,sales executive,1,divorced,48620.0,2137,1,No,25,4.0,3,3,24.0,2,3.0,24,15,7,1977,Yes,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1069,19350,56,No,travel rarely,1443.0,11,5,life sciences,4,Female,89,2,2,sales executive,1,married,48620.0,20328,4,No,16,3.0,3,1,10.0,3,3.0,0,0,0,1967,Yes,A
1070,19360,39,No,travel rarely,867.0,9,2,life sciences,1,Male,87,3,2,manufacturing director,1,married,51510.0,12315,1,No,25,4.0,4,1,10.0,3,3.0,10,7,9,1984,Yes,A
1073,19440,27,Yes,travel rarely,1337.0,22,3,life sciences,1,Female,58,2,1,human resources,2,married,28630.0,19555,1,No,12,3.0,1,0,10.0,2,3.0,1,0,0,1996,No,A
1074,19610,38,No,travel rarely,1404.0,1,3,life sciences,1,Male,59,2,1,sales representative,1,single,28580.0,11473,4,No,14,3.0,1,0,20.0,3,2.0,1,0,0,1985,Yes,A


In [44]:
grupo_a['attrition'].value_counts()


attrition
No     332
Yes     83
Name: count, dtype: int64

In [53]:


a_convertido = round ((83 / 415), 2) # 2 porcentaje de rotación 
a_convertido

0.2

In [37]:
# Filtrar el DataFrame para obtener solo los datos del grupo 'A'
grupo_b = df_ab_testing[df_ab_testing['group_AB_Testing'] == 'B']
grupo_b

,employeenumber,age,attrition,businesstravel,dailyrate,distancefromhome,education,educationfield,environmentsatisfaction,gender,hourlyrate,jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyincome,monthlyrate,numcompaniesworked,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearssincelastpromotion,yearswithcurrmanager,datebirth,remotework,group_AB_Testing
0,1620,51,No,travel rarely,684.0,6,3,life sciences,1,Male,51,3,5,research director,3,married,195370.0,6462,7,No,13,3.0,3,0,10.0,5,3.0,20,15,15,1972,Yes,B
1,2590,52,No,travel rarely,699.0,1,4,life sciences,3,Male,65,2,5,manager,3,married,199990.0,5678,0,No,14,3.0,1,1,34.0,5,3.0,33,11,9,1971,Yes,B
2,3190,42,No,travel rarely,532.0,4,2,technical degree,3,Male,58,3,5,manager,4,married,192320.0,4933,1,No,11,3.0,4,0,22.0,3,2.8,22,11,15,1981,Yes,B
3,20681,47,No,travel rarely,359.0,2,4,medical,1,Female,82,3,4,research director,3,married,171690.0,26703,3,No,19,3.0,2,2,10.0,2,2.8,20,5,6,1976,No,B
4,19000,48,No,travel rarely,117.0,22,3,medical,4,Female,58,3,4,manager,4,married,171740.0,2437,3,No,11,3.0,2,1,10.0,3,3.0,22,4,7,1975,Yes,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1072,19430,42,No,travel rarely,419.0,12,4,marketing,2,Male,77,3,2,sales executive,4,divorced,48620.0,2900,3,No,12,3.2,3,2,14.0,4,3.0,0,0,0,1981,No,B
1075,19660,32,No,travel rarely,1373.0,5,4,life sciences,4,Male,56,2,2,manufacturing director,4,married,48620.0,10138,8,No,24,4.0,2,0,10.0,1,3.0,1,0,0,1991,Yes,B
1076,19740,31,No,travel rarely,1276.0,2,1,medical,4,Female,59,1,1,laboratory technician,4,married,11290.0,17536,1,No,11,3.0,3,3,10.0,4,3.0,1,0,0,1992,Yes,B
1077,20120,40,No,travel rarely,543.0,44,4,life sciences,1,Male,83,3,1,laboratory technician,4,married,48620.0,4060,8,No,19,3.0,3,2,8.0,3,2.0,1,0,0,1983,Yes,B


In [48]:
grupo_b['attrition'].value_counts()

attrition
No     571
Yes     94
Name: count, dtype: int64

In [52]:
b_convertido = round ((94 / 665), 2) # 2 porcentaje de rotación 
b_convertido

0.14